In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

DATA_PATH = "data.json"

def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    z = np.array(data["mapping"])
    
    return X, y, z

def prepare_datasets(test_size, validation_size):
    X, y,z = load_data(DATA_PATH)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test

#architecture for instrument recogition using CNN
def build_model(input_shape):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True)) #sequence to sequence
    model.add(keras.layers.LSTM(64))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(6, activation='softmax'))
    return model

def predict(model, X, y, z):
    X = X[np.newaxis, ...]
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis=1)
    print("Expected instrument: {}, predicted instrument: {}".format(z[y], z[predicted_index]))
    
if __name__ == "__main__":
    X,y,z = load_data(DATA_PATH)
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_model(input_shape)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    # model.summary()
    model.fit(X_train, y_train, validation_data = (X_validation, y_validation), batch_size=32, epochs=30)
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test error: {test_error}, test accuracy: {test_accuracy}")
    



Epoch 1/30


452/452 [==============================] - 18s 26ms/step - loss: 1.4465 - accuracy: 0.4375 - val_loss: 1.0448 - val_accuracy: 0.6382
Epoch 2/30
452/452 [==============================] - 11s 24ms/step - loss: 0.9636 - accuracy: 0.6647 - val_loss: 0.8437 - val_accuracy: 0.7068
Epoch 3/30
452/452 [==============================] - 13s 29ms/step - loss: 0.8098 - accuracy: 0.7288 - val_loss: 0.7732 - val_accuracy: 0.7347
Epoch 4/30
452/452 [==============================] - 13s 29ms/step - loss: 0.7309 - accuracy: 0.7550 - val_loss: 0.7245 - val_accuracy: 0.7530
Epoch 5/30
452/452 [==============================] - 13s 29ms/step - loss: 0.6831 - accuracy: 0.7714 - val_loss: 0.6955 - val_accuracy: 0.7560
Epoch 6/30
452/452 [==============================] - 13s 29ms/step - loss: 0.6522 - accuracy: 0.7858 - val_loss: 0.6732 - val_accuracy: 0.7621
Epoch 7/30
452/452 [==============================] - 13s 28ms/step - loss: 0.6230 - accuracy: 0.7939 - val_loss: 0.6542 - val_accur

In [2]:
def prediction(model, X, y, z):
    X = X[np.newaxis, ...]
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis=1)
    return z[predicted_index]

count = 0
for i in range(len(X_test)):
    x_to_predict = X_test[i]
    y_to_predict = y_test[i]
    z_expected = z[y_to_predict]
    z_predicted = prediction(model, x_to_predict, y_to_predict, z)
    if z_expected != z_predicted:
        count += 1
        print("Expected instrument: {}, predicted instrument: {}".format(z_expected, z_predicted))
print("Total wrong predictions: {}".format(count))
print("In terms of percentage: {}".format(count/len(X_test)*100))

1/1 [==============================] - 0s 39ms/step
Expected instrument: Santur, predicted instrument: ['Tar']
1/1 [==============================] - 0s 39ms/step
Expected instrument: Ud, predicted instrument: ['Setar']
1/1 [==============================] - 0s 42ms/step
Expected instrument: Ney, predicted instrument: ['Setar']
1/1 [==============================] - 0s 35ms/step
Expected instrument: Ud, predicted instrument: ['Setar']
1/1 [==============================] - 0s 44ms/step
Expected instrument: Setar, predicted instrument: ['Ney']
1/1 [==============================] - 0s 44ms/step
Expected instrument: Santur, predicted instrument: ['Ud']
1/1 [==============================] - 0s 36ms/step
Expected instrument: Kamancheh, predicted instrument: ['Tar']
1/1 [==============================] - 0s 48ms/step
Expected instrument: Santur, predicted instrument: ['Ud']
1/1 [==============================] - 0s 46ms/step
Expected instrument: Ney, predicted instrument: ['Setar']
1/1 [==